## 十一、小参数量微调

<img src="peft_model.png" style="margin-left: 0px" width="600px">

- 定义微调数据集加载器
- 定义数据处理函数
- 加载预训练模型：AutoModel.from_pretrained(MODEL_NAME_OR_PATH)
- 在预训练模型上增加任务相关输出层 （如果需要）
- 加载预训练 Tokenizer：AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
- **定义注入参数的方法（见下文）**
- 定义各种超参
- 定义 Trainer
- 定义 Evaluation Metric
- 开始训练

### 11.1、Prompt Tuning

- 在输入序列前，额外加入一组伪 Embedding 向量
- 只训练这组伪 Embedding，从而达到参数微调的效果

<img src="soft-prompt.png" style="margin-left: 0px" width="600px">

### 11.2、P-Tuning

- 用一个生成器生成上述伪 Embedding
- 只有生成器的参数是可训练的

<img src="pt.png" style="margin-left: 0px" width="600px">

### 11.3、Prefix-Tuning

- 伪造前面的 Hidden States
- 只训练伪造的这个 Prefix

<img src="pt2.png" style="margin-left: 0px" width="600px">

### 11.4、LoRA

- 在 Transformer 的参数矩阵上加一个低秩矩阵（$A\times B$）
- 只训练 A，B
- 理论上可以把上述方法应用于 Transformer 中的任意参数矩阵，包括 Embedding 矩阵
- 通常应用于 Query, Value 两个参数矩阵

<img src="lora.png" style="margin-left: 0px" width="600px">

### 11.5、QLoRA

什么是模型量化

<img src="float.png" style="margin-left: 0px" width="600px">
<img src="quant.png" style="margin-left: 0px" width="600px">

更多参考: https://huggingface.co/blog/hf-bitsandbytes-integration

QLoRA 引入了许多创新来在不牺牲性能的情况下节省显存：

- 4位 NormalFloat（NF4），一种对于正态分布权重而言信息理论上最优的新数据类型
- 双重量化，通过量化量化常数来减少平均内存占用
- 分页优化器，用于管理内存峰值

原文实现：单个48G的GPU显卡上微调65B的参数模型，保持16字节微调任务的性能

### 11.6、AdaLoRA

- 不预先指定可训练矩阵的秩
- 根据参数矩阵的重要性得分，在参数矩阵之间自适应地分配参数预算。


## 十二、**实战** 

基于ChatGLM2或Llama2， 微调一个同时具有NLU和问答能力对话机器人

### 12.1、数据源

酒店预订场景
https://github.com/thu-coai/CrossWOZ

酒店数据库
https://github.com/thu-coai/CrossWOZ/blob/master/data/crosswoz/database/hotel_db.json

### 12.2、数据增强

- 从CrossWOZ数据集中抽取了只关于酒店的对话
- 利用ChatGPT进行如下修改和补充
    - 对设施的描述更口语化：“找一家有国际长途电话的酒店” -> “找一家能打国际长途的酒店”
    - 补充一定比例的多轮问答，和结束语对话（p=0.3）
    - 补充按酒店名（简称）、价格上限查询的对话（原数据中没有这类说法）
 
最终按8:1:1拆分训练集、验证集和测试集

**样本举例**

```json
[
    {
        "role": "user",
        "content": "请帮我找一个评分是4.5分以上的酒店。"
    },
    {
        "role": "search",
        "arguments": {
            "rating_range_lower": 4.5
        }
    },
    {
        "role": "return",
        "records": [
            {
                "name": "北京天伦王朝酒店",
                "type": "豪华型",
                "address": "北京东城区王府井大街50号",
                "subway": "灯市口地铁站A口",
                "phone": "010-58168888",
                "facilities": [
                    "公共区域和部分房间提供wifi",
                    "宽带上网",
                    "国际长途电话",
                    "吹风机",
                    "24小时热水",
                    "西式餐厅",
                    "中式餐厅",
                    "残疾人设施",
                    "室内游泳池",
                    "会议室",
                    "健身房",
                    "无烟房",
                    "商务中心",
                    "桑拿",
                    "棋牌室",
                    "早餐服务",
                    "接站服务",
                    "接机服务",
                    "接待外宾",
                    "洗衣服务",
                    "行李寄存",
                    "租车",
                    "叫醒服务"
                ],
                "price": 815.0,
                "rating": 4.5,
                "hotel_id": 4
            }
        ]
    },
    {
        "role": "assistant",
        "content": "推荐您去北京天伦王朝酒店。"
    },
    {
        "role": "user",
        "content": "这家酒店的价格是多少？"
    },
    {
        "role": "assistant",
        "content": "北京天伦王朝酒店的价格是每晚815元。"
    },
    {
        "role": "user",
        "content": "好的，我决定入住北京天伦王朝酒店了。"
    },
    {
        "role": "assistant",
        "content": "好的，祝您入住愉快！"
    }
]
```

### 12.2、ChatGLM2 的数据加载方式分析

输入本质上就是一段按格式拼接的文本

In [1]:
# https://huggingface.co/THUDM/chatglm2-6b/blob/main/tokenization_chatglm.py
# history = [("你好","你好，我是AI"),("你多大","我有6B参数")]
def build_prompt(self, query, history=None):
    if history is None:
        history = []
    prompt = ""
    for i, (old_query, response) in enumerate(history):
        prompt += "[Round {}]\n\n问：{}\n\n答：{}\n\n".format(i + 1, old_query, response)
    prompt += "[Round {}]\n\n问：{}\n\n答：".format(len(history) + 1, query)
    return prompt

    
# https://github.com/THUDM/ChatGLM2-6B/blob/main/ptuning/main.py
def preprocess_function_eval(examples):
    inputs, targets = [], []
    for i in range(len(examples[prompt_column])):
        if examples[prompt_column][i] and examples[response_column][i]:
            query = examples[prompt_column][i]
            history = examples[history_column][i] if history_column is not None else None
            prompt = tokenizer.build_prompt(query, history)
            inputs.append(prompt)
            targets.append(examples[response_column][i])

    inputs = [prefix + inp for inp in inputs]
    model_inputs = tokenizer(inputs, max_length=data_args.max_source_length, truncation=True, padding=True)
    labels = tokenizer(text_target=targets, max_length=max_target_length, truncation=True)

    if data_args.ignore_pad_token_for_loss:
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

def preprocess_function_train(examples):
    max_seq_length = data_args.max_source_length + data_args.max_target_length + 1

    model_inputs = {
        "input_ids": [],
        "labels": [],
    }
    for i in range(len(examples[prompt_column])):
        if examples[prompt_column][i] and examples[response_column][i]:
            query, answer = examples[prompt_column][i], examples[response_column][i]

            history = examples[history_column][i] if history_column is not None else None
            prompt = tokenizer.build_prompt(query, history)

            prompt = prefix + prompt
            a_ids = tokenizer.encode(text=prompt, add_special_tokens=True, truncation=True,
                                     max_length=data_args.max_source_length)

            b_ids = tokenizer.encode(text=answer, add_special_tokens=False, truncation=True,
                                     max_length=data_args.max_target_length)

            context_length = len(a_ids)
            input_ids = a_ids + b_ids + [tokenizer.eos_token_id]
            labels = [tokenizer.pad_token_id] * context_length + b_ids + [tokenizer.eos_token_id]
            
            pad_len = max_seq_length - len(input_ids)
            input_ids = input_ids + [tokenizer.pad_token_id] * pad_len
            labels = labels + [tokenizer.pad_token_id] * pad_len
            if data_args.ignore_pad_token_for_loss:
                labels = [(l if l != tokenizer.pad_token_id else -100) for l in labels]

            model_inputs["input_ids"].append(input_ids)
            model_inputs["labels"].append(labels)

    return model_inputs`


SyntaxError: invalid syntax (3693428564.py, line 70)

### 12.2.1、**知识点 1：**数据是怎么拼接的

<img src="batch.png" width=400px/>

### 12.2.2、**知识点 2：**为什么训练时和测试时数据拼接方式不同


<div class="alert alert-warning">
<b>思考：</b> Batch 拼接时 Padding 在哪边
</div>

<img src="training-batch.png" width=400px/>

<img src="inference-batch.png" width=400px/>


### 12.3、结果的评价方式


```python
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if data_args.ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    score_dict = {
        "rouge-1": [],
        "rouge-2": [],
        "rouge-l": [],
        "bleu-4": []
    }
    for pred, label in zip(decoded_preds, decoded_labels):
        hypothesis = list(jieba.cut(pred))
        reference = list(jieba.cut(label))
        rouge = Rouge()
        scores = rouge.get_scores(' '.join(hypothesis), ' '.join(reference))
        result = scores[0]

        for k, v in result.items():
            score_dict[k].append(round(v["f"] * 100, 4))
        bleu_score = sentence_bleu([list(label)], list(
            pred), smoothing_function=SmoothingFunction().method3)
        score_dict["bleu-4"].append(round(bleu_score * 100, 4))

    for k, v in score_dict.items():
        score_dict[k] = float(np.mean(v))
    return score_dict
```


### 12.3.1、**知识点 3：**为什么不直接看 Loss

语言模型在训练过程中 Loss 是如何计算的

<img src="lm_loss.png" style="margin-left: 0px" width="600px">

考虑下面的情况：

<img src="lm_loss2.png" style="margin-left: 0px" width="400px">

相似度高，但是 loss 大，所以 loss 没法准确反应出我们真实的预期

### 12.3.2、**知识点 4：**生成模型常用的评价方法

- BLEU Score:
  - 计算输出与参照句之间的 n-gram 准确率（n=1...4）
  - 对短输出做惩罚
  - 在整个测试集上平均下述值

$\mathrm{BLEU}_4=\min\left(1,\frac{output-length}{reference-length}\right)\left(\prod_{i=1}^4 precision_i\right)^{\frac{1}{4}}$

- Rouge Score:

  - Rouge-N：将模型生成的结果和标准结果按 N-gram 拆分后，只计算召回率；
  - Rouge-L: 利用了最长公共子序列（Longest Common Sequence），计算：$P=\frac{LCS(c,r)}{len(c)}$, $R=\frac{LCS(c,r)}{len(r)}$, $F=\frac{(1+\beta^2)PR}{R+\beta^2P}$

- 对比 BLEU 与 ROUGE
  - BLEU 能评估流畅度，但指标偏向于较短的翻译结果（brevity penalty 没有想象中那么强）
  - ROUGE 不管流畅度，所以只适合深度学习的生成模型：结果都是流畅的前提下，ROUGE 反应参照句中多少内容被生成的句子包含（召回）

<div class="alert alert-warning">
<b>思考：</b> 为什么不能直接用BLEU或者ROUGE作为Loss函数？
</div>
 
### 12.3.3、编写训练代码

见附件

### 12.3.4、测试集效果

| Model       | Method      | BLEU-4 | SLOT-P | SLOT-R | SLOT-F1 | 
| :---------- | :---------- | :----: | :-----: | :-----: | :-----: | 
| ChatGLM2-6B  | P-Tuning V2 | 61.44 |  93.68 |  90.51  |  92.07  |   
|             | LoRA        | **63.45**  |  94.32  |  93.41  |  93.86  |   
| Llama2-7B | QLoRA | 63.05 | **95.96** |  **95.50**  |  **95.73**  | 

### 12.3.5、训练过程

**ChatGLM 2 (6B) + P-Tuning V2**

<img src="log-chatglm2-pt2.png" style="margin-left: 0px" width="600px">

**ChatGLM 2 (6B) + LoRA**

<img src="log-chatglm2-lora.png" style="margin-left: 0px" width="600px">

**Llama 2 (7B) + QLoRA**

<img src="log-llama2-qlora.png" style="margin-left: 0px" width="600px">

## 十五、即想要开放能力又想要垂直能力怎么办

**大模型的训练过程（宏观）**

<img src="train_llm.png" style="margin-left: 0px" width="600px">

**混合通用领域和垂直领域数据（举例：度小满--轩辕）**

<img src="xuanyuan.png" style="margin-left: 0px" width="600px">


## 十六、数据准备与处理

### 16.1、数据采集

- 自然来源（如业务日志）：真实数据
- Web 抓取：近似数据
- 人造

### 16.2、数据标注

- 专业标注公司
  - 定标准，定验收指标
  - 预标注
  - 反馈与优化
  - 正式标注
  - 抽样检查：合格->验收；不合格->返工
- 众包
  - 定标准，定检验指标
  - 抽样每个工作者的质量
  - 维系高质量标注者社区
- 主动学习：通过模型选择重要样本，由专家标注，再训练模型
- 设计产品形态，在用户自然交互中产生标注数据（例如点赞、收藏）

### 16.3、数据清洗

- 去除不相关数据
- 去除冗余数据（例如重复的样本）
- 去除误导性数据（业务相关）

### 16.4、样本均衡性

- 尽量保证每个标签（场景/子问题）都有足够多的训练样本
- 每个标签对应的数据量尽量相当
  - 或者在保证每个标签样本充值的前提下，数据分布尽量接近真实业务场景的数据分布
- 数据不均衡时的策略
  - 数据增强：为数据不够类别造数据：（1）人工造；（2）通过模板生成再人工标注；（3）由模型自动生成（再人工标注/筛选）
  - 数据少的类别数据绝对数量也充足时，Downsample 一般比 Upsample 效果好
  - 实在没办法的话，在训练 loss 里加权（一般不是最有效的办法）
- 根据业务属性，保证其他关键要素的数据覆盖，例如：时间因素、地域因素、用户年龄段等

### 16.5、数据集构建

- 数据充分的情况下
  - 切分训练集（训练模型）、验证集（验证超参）、测试集（检验最终模型+超参的效果）
  - 以随机采样的方式保证三个集合的数据分布一致性
  - 在以上三个集合里都尽量保证各个类别/场景的数据覆盖
- 数据实在太少
  - 交叉验证


## 作业：训练自己的模型

请跟随[实验指导](../lab/index.ipynb)完成本次作业。